In [3]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colormaps
import ipywidgets as widgets
from IPython.display import display

In [4]:
images_PMV = [[open(f'pmv-seoul-D{d:02}-H{h:02}.png', 'rb').read() for h in range(24)] for d in range(10)]
assert len(images_PMV), 'Failed to load the PMV images!'

In [5]:
w_label_dict  = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun', 'W.day_avg', 'W.end_avg', 'W.tot_avg']
w_color_dict  = ['darkviolet', 'orange', 'skyblue', 'green', 'peru', 'blue', 'red', 'midnightblue', 'deeppink', 'black']

In [6]:
L_dh = np.load('L_sudo_weekly.npy')
G_dhj = np.load('Gj_sudo_weekly.npy')
G_dh = G_dhj.max(axis=2)

In [7]:
w_label_dict  = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun', 'W.day_avg', 'W.end_avg', 'W.tot_avg']
w_color_dict  = ['rebeccapurple', 'orangered', 'lightseagreen', 'forestgreen', 'peru', 'blue', 'red', 'midnightblue', 'maroon', 'black']


In [8]:

def parametrize_highlight(d, d0):
    flag = d==d0
    return dict(
        alpha = (1 if flag else 0.25),
        zorder = (100 if flag else d),
        linewidth = (2 if flag else 1.5),
        marker = ('s' if flag else ('.' if d<7 else '+'))
    )

def plot_AG_seperated(d_this):
    fig, axes = plt.subplots(ncols=2, figsize=(14,6))
    
    fontsize=14
    axes[0].set_title('Anisotropy Λ by hour', fontsize=fontsize)
    axes[1].set_title('Centripetality Γ by hour', fontsize=fontsize)
    axes[0].set_xlabel('Departing Hour', fontsize=fontsize-2)
    axes[1].set_xlabel('Departing Hour', fontsize=fontsize-2)
    axes[0].set_ylabel('Λ')
    axes[1].set_ylabel('Γ')

    for d in range(10):
        params = dict(color=w_color_dict[d], **parametrize_highlight(d,d_this))
        y11 = axes[0].plot(range(24), L_dh[d,:], **params)
        y12 = axes[1].plot(range(24), G_dh[d,:], **params, label=w_label_dict[d])
    
    for ax in axes:
        ax.grid()
        ax.axvline(8,  color='k', alpha=0.5)
        ax.axvline(18, color='k', alpha=0.5)
    fig.legend(loc='lower left', bbox_to_anchor=(-0.03, 0.1))
    # plt.close(fig)
    plt.show()

In [9]:
def plot_AG_combined(d_this):
    fig, ax = plt.subplots(figsize=(6,6))
    ax.set_title('The (Λ,Γ) space')
    ax.set_xlabel('Λ, Anisotropy')
    ax.set_ylabel('Γ, Centripetality')
    
    ax.set_xlim(0.2, 0.6)
    ax.set_ylim(0.45, 0.85)
    
    ax.set_aspect('equal')
    ax.grid()
    
    for d in range(10):
        ax.plot(L_dh[d], G_dh[d], '.-', color='gray', alpha=0.25)

    L_h, G_h = L_dh[d_this], G_dh[d_this]
    
    ax.plot(L_h, G_h, '-', color='k', zorder=1)
    ax.scatter(L_h, G_h, c=colormaps['hsv'](np.linspace(1,0,24)), zorder=2)
    
    props = dict(boxstyle='round', fc='black', lw=0, alpha=0.7)
    for h in [0, 7, 17]:
        ax.text(L_h[h], G_h[h], f'{h}', ha='center', color='white', bbox=props)
    ax.text(0.41, 0.82, f'day of week : {w_label_dict[d]}', ha='left', color='gray')

    # plt.close(fig)
    plt.show()

# PMV $T_i$

In [ ]:
d0, h0 = 0, 8

radio_weekday = widgets.RadioButtons(
    options=w_label_dict,
    value=w_label_dict[d0],
    description='Day of weeks for defining the mobility data (Tij)',
    disabled=False
)
slider_hours = widgets.IntSlider(min=0, max=23, value=h0, description='Hour')
imager_PMV = widgets.Image(value=images_PMV[d0][h0], format='png', width=800)
output_AG_seperated = widgets.Output()
output_AG_combined = widgets.Output()

# display(
#     widgets.VBox([
#         widgets.HBox([widgets.VBox([radio_weekday, slider_hours]), imager_PMV]),
#         widgets.HBox([output_AG_seperated, output_AG_combined])
#     ])
# )

display(widgets.HBox([widgets.VBox([radio_weekday, slider_hours]), imager_PMV]))
display(widgets.HBox([output_AG_seperated, output_AG_combined]))

def handle_slider_hour_change(change):
    d = w_label_dict.index(radio_weekday.value)
    h = change['new']
    imager_PMV.value = images_PMV[d][h]

def handle_radio_weekday_change(change):
    d = w_label_dict.index(change['new'])
    h = slider_hours.value
    imager_PMV.value = images_PMV[d][h]
    with output_AG_seperated:
        output_AG_seperated.clear_output()
        plot_AG_seperated(d)
    with output_AG_combined:
        output_AG_combined.clear_output()
        plot_AG_combined(d)

slider_hours.observe(handle_slider_hour_change, names='value')
radio_weekday.observe(handle_radio_weekday_change, names='value')

handle_radio_weekday_change(dict(new=w_label_dict[d0]))